In [2]:
# jupyter notebook with an example for an interferometer calculation
import sympy as sy
from sympy import I
from IPython.display import Latex
sy.init_printing()

from Interferometry import Interferometer, Hamiltonian, Pulse, U

# define symbolic variables
m1,m2,g1,g2,k,T,m,gmag,ggrav,wtrap,gravgrad1,gravgrad2,xtrap=sy.symbols("m1,m2,g1,g2,k,T,m,g_m,g,Ω,γ1,γ2,x_\Omega")



Define Hamiltonians by passing a parameter list [a,b,c,d,e,f]
 corresponding to the Hamiltonian
 $H=a p^2 + b p +c(px+xp) + d x + e + f x^2$ 
 

In [3]:
H=Hamiltonian([1/(2*m), 0,0,m*gmag+m*ggrav-m*wtrap**2*xtrap,m/2*wtrap**2 *xtrap**2,m/2*wtrap**2])

print('Hamiltonian is:')         
print(H)

Hamiltonian is:


Define upper and lower sequence (note the ordering of operators is the same as if acting on a initial ket state)

In [4]:
upperSequence=[U(H,T),  Pulse(-k), U(H,T), Pulse(k)] 
lowerSequence=[Pulse(-k),  U(H,T),  Pulse(k), U(H,T)]

Initialize an interferometer object with the upper and lower sequence

In [5]:
I=Interferometer(upperSequence, lowerSequence)

#calculate overlap operator
resdict,res=I.phase()
res

In [6]:
# Clean up and expand everything in interferometer duration
order=5
expandvar=T

resultBeautifyer = lambda x: sy.expand(sy.series(x,expandvar,n=order))

result=[resultBeautifyer(res.a),
        resultBeautifyer(res.b),
        resultBeautifyer(res.c),
        resultBeautifyer(res.d),
        resultBeautifyer(res.e),
        resultBeautifyer(res.f)]

# Show me the operator coefficients of exp(i O(x,p))
Latex(f"""
\\begin{{align}}
    &\\hat{{p}}^2 & {{{sy.latex(result[0])}}} \\\\
    &\\hat{{p}} & {{{sy.latex(result[1])}}} \\\\
    &\\hat{{p}}\\hat{{x}}+\\hat{{x}}\\hat{{p}} & {{{sy.latex(result[2])}}} \\\\
    &\\hat{{x}} & {{{sy.latex(result[3])}}} \\\\
    &\\hat{{x}}^2 & {{{sy.latex(result[5])}}}\\\\
    &\\hat{{1}} & {{{sy.latex(result[4])}}}
\\end{{align}}
""")

<IPython.core.display.Latex object>

In [7]:
sp=result[1]*sy.symbols('\hat{p}')
sx=result[3]*sy.symbols('\hat{x}')
sphi=result[4]
# Show me the operator part
display(sy.expand(sp+sx))
# Show me the non-operator part
display(sphi)

In [8]:
# Just gimme the result for comparison with Matthias's expressions
from sympy import I

# Define magnetic gradients
mub=sy.symbols("\mu_{B}")
gfj=sy.symbols("g_{F_j}") 
mfj=sy.symbols("m_{F_j}")
x = sy.symbols("x")
Bx=sy.Function('B_x')(x)
r=sy.Mul(sy.Mul(sy.Mul(mub,gfj),mfj),sy.diff(Bx,x)).subs(x,"x_0")

In [10]:
# Give me the result for vanishing state dependent accelerations
print('No state-dependent linear magnetic potential')
display(sy.collect(sy.expand((sp+sx+sphi)/I).subs(gmag,0),k*wtrap**2*T**2))

# Give me the result for non-vanishing state dependent accelerations (substituted and without)
print('Only state-dependent linear magnetic potential')
display(sy.collect(sy.expand((sp+sx+sphi)/I),k*wtrap**2*T**2))
print('Substituted definition of state-dependent linear potential')
display(sy.collect(sy.expand((sp+sx+sphi)/I).subs(gmag,r/m),k*wtrap**2*T**2))

No state-dependent linear magnetic potential


Only state-dependent linear magnetic potential


Substituted definition of state-dependent linear potential
